In [6]:
import trimesh
from trimesh.visual.texture import TextureVisuals, SimpleMaterial
import open3d as o3d
from PIL import Image
import numpy as np
import pyrender
import math
from tqdm import tqdm
import os
from utils_fish import fps_sample_vertices, visualize_sorted_vertices_plotly, visualize_direction_vec_w_fish, get_rotation_matrix_from_vectors, rotate_mesh_vertices, rotation_matrix_axis_angle, rotation_matrix_from_vectors, rotate_mesh, get_fish_forward_vector

In [3]:

# ----------------------------------------
# Basic parameters
# ----------------------------------------
DATA_DIR      = './3d_fish_reduced'
INPUT_MESH    = 'reference_fish_smpl.glb'
SCALE_RATIO   = 0.06
NUM_INSTANCES = 360


# original mesh load and scale
reference_fish = trimesh.load(os.path.join(DATA_DIR, INPUT_MESH))
reference_fish = reference_fish.dump()
reference_fish = trimesh.util.concatenate(reference_fish)

In [4]:
# fps sampling 
points = fps_sample_vertices(reference_fish, max_samples=NUM_INSTANCES)

In [8]:
# Normalize scale of fishes 
import pandas as pd 
file_path = "fish_asset_color_ranking_kmeans.csv"
small_fishes = []


df = pd.read_csv(file_path)
file_names = [os.path.splitext(name)[0]+"_smpl.glb" for name in df["file_name"]]
file_names = [os.path.join(DATA_DIR, name) for name in file_names]
for i, file in enumerate(file_names):
    small_fish = trimesh.load(file)
    small_fish = small_fish.dump()
    small_fish = trimesh.util.concatenate(small_fish)
    small_fishes.append(small_fish)
    # small_fish.show()


In [29]:
forward_vec = get_fish_forward_vector(reference_fish)  # 머리 → 꼬리 방향 벡터

In [45]:
idx = 300
specific_fish = small_fishes[idx]
dir_vec = get_fish_forward_vector(specific_fish)  #
visualize_direction_vec_w_fish(specific_fish, dir_vec)

In [40]:
visualize_direction_vec_w_fish(reference_fish, forward_vec)

In [50]:
from trimesh.registration import icp
ref_points = reference_fish.vertices
small_fishes_aligned = []
for small_fish in tqdm(small_fishes):
    # 2. Source Mesh 로드 및 Vertex 추출
    small_fish_points = small_fish.vertices

    # 3. ICP 정렬 (point-to-point)
    matrix, _, cost = icp(
        small_fish_points,
        ref_points,
    )

    # 4. 변환 행렬 적용
    small_fish.apply_transform(matrix)

    # 5. 저장
    small_fishes_aligned.append(small_fish)

 20%|██        | 73/360 [2:20:31<9:12:29, 115.50s/it] 


KeyboardInterrupt: 

In [10]:
# Place small fish instances
scene = trimesh.Scene()
sorted_indices = np.argsort(points[:, 0])
sorted_points = points[sorted_indices]

for small_fish, p in tqdm(zip(small_fishes, sorted_points), desc='Instance Placement'):
    small_fish.apply_scale(SCALE_RATIO)
    small_fish.apply_translation(p)
    scene.add_geometry(small_fish)
scene.export('school_of_different_fishes_aligned_reduced.glb')

Instance Placement: 360it [00:02, 131.29it/s]


In [27]:
for i, small_fish in tqdm(enumerate(small_fishes), desc="Aligning Small Fish"):
    src_dir = get_fish_forward_vector(small_fish)
    R_mat = get_rotation_matrix_from_vectors(src_dir, forward_vec)

    small_fishes[i] = rotate_mesh_vertices(small_fish, R_mat)

Aligning Small Fish: 360it [08:05,  1.35s/it]


In [ ]:
# Place small fish instances
scene = trimesh.Scene()
sorted_indices = np.argsort(points[:, 0])
sorted_points = points[sorted_indices]

for small_fish, p in tqdm(zip(small_fishes, sorted_points), desc='Instance Placement'):
    small_fish.apply_scale(SCALE_RATIO)
    small_fish.apply_translation(p)
    scene.add_geometry(small_fish)
scene.export('school_of_different_fishes_aligned.glb')

Instance Placement: 360it [00:22, 16.25it/s]
